# <span style="color:red">1. Import libs</span>

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import NullFormatter
import datetime as dt
from math import sqrt, pow
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# warnings
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

%matplotlib inline

# <span style="color:red">2. Read CSV</span>

In [ ]:
df_2013 = pd.read_csv("data/2013_Iowa_Liquor_Sales.csv", low_memory=False)

# <span style="color:red">3. Format data</span>

In [ ]:
df_2013['Date'] = pd.to_datetime(df_2013['Date'])
#df_2013['Year'] = df_2013['Date'].dt.strftime('%Y')
df_2013['Month'] = df_2013['Date'].dt.strftime('%m')
#df_2013['Day'] = df_2013['Date'].dt.strftime('%d')
df_2013['Day Of Week'] = df_2013['Date'].dt.day_name()

In [ ]:
df_2013['Month'] = df_2013['Month'].replace({'01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May', '06': 'June', 
                                            '07':'Jul', '08': 'Aug','09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'})

In [ ]:
df_2013['Store Number'] = df_2013['Store Number'].astype('str')
df_2013['County Number'] = df_2013['County Number'].astype('str')
df_2013['Category'] = df_2013['Category'].astype('str')
df_2013['Item Number'] = df_2013['Item Number'].astype('str')
df_2013['Vendor Number'] = df_2013['Vendor Number'].astype('str')

In [ ]:
month_category = CategoricalDtype(categories=["Jan", "Feb", "Mar", "Apr", "May", "June", "Jul", "Aug",
                                            "Sep", "Oct", "Nov", "Dec"], ordered=True)

dayOfWeek_category = CategoricalDtype(categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", 
                                                  "Saturday", "Sunday"], ordered=True)

# <span style="color:red">4. Functions</span>

#### <span style="color:blue">Turns large tick values (in the billions, millions and thousands) such as 4500 into 4.5K and also appropriately turns 4000 into 4K (no zero after the decimal).</span>

In [ ]:
def reformat_large_tick_values(tick_val, pos):
    if tick_val >= 1000000000:
        val = round(tick_val/1000000000, 1)
        new_tick_format = '{:}B'.format(val)
    elif tick_val >= 1000000:
        val = round(tick_val/1000000, 1)
        new_tick_format = '{:}M'.format(val)
    elif tick_val >= 1000:
        val = round(tick_val/1000, 1)
        new_tick_format = '{:}K'.format(val)
    elif tick_val < 1000:
        new_tick_format = round(tick_val, 1)
    else:
        new_tick_format = tick_val

    # make new_tick_format into a string value
    new_tick_format = str(new_tick_format)
    
    # code below will keep 4.5M as is but change values such as 4.0M to 4M since that zero after the decimal isn't needed
    index_of_decimal = new_tick_format.find(".")
    
    if index_of_decimal != -1:
        value_after_decimal = new_tick_format[index_of_decimal+1]
        if value_after_decimal == "0":
            # remove the 0 after the decimal point since it's not needed
            new_tick_format = new_tick_format[0:index_of_decimal] + new_tick_format[index_of_decimal+2:]
            
    return new_tick_format

#### <span style="color:blue">Find missing dates between 2 dates</span>

In [ ]:
def find_missing_dates(df, sdate, edate):
    dt_range = pd.date_range(start= sdate, end= edate, freq='B')
    return dt_range.difference(df['Date'])

#### <span style="color:blue">Check if a date exists</span>

In [ ]:
def exist_date(df, full_date):
    return (df['Date'] == pd.Timestamp(full_date)).any()

#### <span style="color:blue">Filter DataFrame Rows on Dates</span>

In [ ]:
# Filter by a single date
def df_BySingleDate(df, date):
    return df[df['Date'].dt.strftime('%Y-%m-%d') == date]

# Filter Rows by Dates in pandas DataFrame
def df_ByDates(df, start_date, end_date):
    return df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

# Filter by single month
def df_BySingleMonth(df, month):
    return df[df['Date'].dt.strftime('%Y-%m') == month]

# Filter by single year
def df_BySingleYear(df, year):
    return df[df['Date'].dt.strftime('%Y') == year]

# Filter data for specific weekday 
def df_ByWeekDay(df, wk_day):
    return df.loc[df['Date'].dt.weekday == wk_day]

#### <span style="color:blue">Filter DataFrame Rows on Store Number, City, County, Category ID, Vendor, Item</span>

In [ ]:
# Filter by Store number
def df_ByStoreNumber(df, store_number):
    return df[df['Store Number'] == store_number]

# Filter by City name
def df_ByCity(df, city_name):
    return df[df['City'] == city_name]

# Filter by County name
def df_ByCounty(df, county_name):
    return df[df['County'] == county_name]

# Filter by Cat id
def df_ByCat(df, cat_id):
    return df[df['Category'] == cat_id]

# Filter by Vendor number
def df_ByVendor(df, vendor_number):
    return df[df['Vendor Number'] == vendor_number]

# Filter by Item number
def df_ByItem(df, item_number):
    return df[df['Item Number'] == item_number]

#### <span style="color:blue">Check if keyword/data exist</span>

In [ ]:
def exist_inputData(keyword):
    return key_word in df_2013.values

#### <span style="color:blue">Total Sale/Invoice</span>

In [ ]:
def total_sale_invoice(df):
    total_sale = '${:,.0f}'.format(df['Sale (Dollars)'].sum())
    total_inv = '{:,.0f}'.format(len(df['Invoice/Item Number']))
    
    return total_sale, total_inv

#### <span style="color:blue">Total Sale/Invoice of Day of Week</span>

In [ ]:
def total_sale_invoice_dow(df):
    dow_sale = df['Sale (Dollars)'].groupby(df['Day Of Week'].astype(dayOfWeek_category)).sum()                         
    dow_sale_fr = dow_sale.apply(lambda x: f'${x:,.0f}').to_frame()

    dow_inv = df['Day Of Week'].astype(dayOfWeek_category).value_counts(sort=False)
    dow_inv_fr = dow_inv.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoices')

    sale_invoice_dow = pd.concat([dow_sale_fr, dow_inv_fr], axis=1, join="inner")
    
    return dow_sale, dow_inv, sale_invoice_dow

In [ ]:
# Day of Week with Best Sales
def plot_dow_sale_inv_graph(df_sale, df_inv):     
    fig, ax1 = plt.subplots(figsize=(7, 4))

    ax2 = ax1.twinx()

    df_sale.plot(kind='bar', color='y', ax=ax1, label = 'Sales (USD)')
    df_inv.plot(kind='line', marker='d', ax=ax2, label='Number of invoices')
     
    ax1.set_ylabel("Sales (USD)", fontsize=13, color='r', labelpad=15)
    ax1.set_xlabel("Day of Week", fontsize=13, color='r', labelpad=15)
    ax1.set_xticklabels(['Mon', 'Tue','Wed','Thu','Fri','Sat','Sun'], rotation = 0)
    
    ax1.yaxis.tick_left()
    ax1.yaxis.set_major_formatter(reformat_large_tick_values)
    ax1.yaxis.set_minor_formatter(NullFormatter())

    ax2.set_ylabel("Number of invoices", fontsize=13, color='r', labelpad=15)
    
    ax2.yaxis.tick_right()
    ax2.yaxis.set_major_formatter(reformat_large_tick_values)
    ax2.yaxis.set_minor_formatter(NullFormatter())

    # combine 2 legends into 1
    lines_1, labels_1 = ax1.get_legend_handles_labels()
    lines_2, labels_2 = ax2.get_legend_handles_labels()

    lines = lines_1 + lines_2
    labels = labels_1 + labels_2

    ax1.legend(lines, labels, loc=0)
    ax2.legend().set_visible(False)

    plt.title("Day of Week with Best Sales", pad=20, fontsize=14); 

#### <span style="color:blue">Total Sale/Invoice of Month</span>

In [ ]:
def total_sale_invoice_month(df):
    month_sale = df['Sale (Dollars)'].groupby(df['Month'].astype(month_category)).sum()
    month_sale_fr = month_sale.apply(lambda x: f'${x:,.0f}').to_frame()

    month_inv = df['Month'].value_counts(sort=False)
    month_inv_fr = month_inv.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoices')

    month_bottle = df['Bottles Sold'].groupby(df['Month'].astype(month_category)).sum()
    month_bottle_fr = month_bottle.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of bottle sold')

    sale_invoice_month = pd.concat([month_sale_fr, month_inv_fr, month_bottle_fr], axis=1, join="inner")

    return month_sale, month_inv, sale_invoice_month

In [ ]:
# Month with Best Sales
def plot_month_sale_inv_graph(df_sale, df_inv):     
    fig, ax1 = plt.subplots(figsize=(7, 4))
    ax2 = ax1.twinx()

    df_sale.plot(kind='bar', color='y', ax=ax1, label='Sales (USD)')
    df_inv.plot(kind='line', marker='d', ax=ax2, label='Number of invoices')

    ax1.set_ylabel("Sales (USD)", fontsize=13, color='r', labelpad=15)
    ax1.set_xlabel("Months", fontsize=13, color='r', labelpad=15)
    ax1.set_xticklabels(['Jan', 'Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], rotation = 0)
    ax1.yaxis.tick_left()
    ax1.yaxis.set_major_formatter(reformat_large_tick_values)
    ax1.yaxis.set_minor_formatter(NullFormatter()) 

    ax2.set_ylabel("Number of invoices", fontsize=13, color='r', labelpad=15)
    ax2.yaxis.tick_right()
    ax2.yaxis.set_major_formatter(reformat_large_tick_values)
    ax2.yaxis.set_minor_formatter(NullFormatter())

    # combine 2 legends into 1
    lines_1, labels_1 = ax1.get_legend_handles_labels()
    lines_2, labels_2 = ax2.get_legend_handles_labels()

    lines = lines_1 + lines_2
    labels = labels_1 + labels_2

    ax1.legend(lines, labels, loc=0)
    ax2.legend().set_visible(False)

    plt.title("Months with Best Sales", pad=20, fontsize=14);

#### <span style="color:blue">Top 10 stores with Best sale/invoices</span>

In [ ]:
def top10_store_sale_inv(df):
    store_sale = df.groupby(['Store Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
    store_sale_fr = store_sale.apply(lambda x: f'${x:,.0f}').to_frame()

    store_inv = df['Store Name'].value_counts().nlargest(10)
    store_inv_fr = store_inv.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoices')

    return store_sale, store_sale_fr, store_inv, store_inv_fr

In [ ]:
def plot_top10_store_sale_graph(df_store_sale):    
    ax = df_store_sale.plot.barh(figsize=(7, 4))

    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("Store Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Sales in (USD)", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 Stores with Best Sales (USD)", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10_store_inv_graph(df_store_inv):    
    ax = df_store_inv.plot.barh(figsize=(7, 4))
    
    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("Store Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Number of invoices", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 Stores with Best number of invoices", fontsize=13, color='r', pad=15);

#### <span style="color:blue">Top 10 cities with Best sale/invoices</span>

In [ ]:
def top10_cities_sale_inv(df):
    city_sale = df.groupby(['City'])['Sale (Dollars)'].sum().nlargest(10)                                   
    city_sale_fr = city_sale.apply(lambda x: f'${x:,.0f}').to_frame()

    city_inv = df['City'].value_counts().nlargest(10)
    city_inv_fr = city_inv.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoices')

    return city_sale, city_sale_fr, city_inv, city_inv_fr

In [ ]:
def plot_top10_city_sale_graph(df_city_sale):
    ax = df_city_sale.plot.barh(figsize=(7, 4))

    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("City Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Sales (USD)", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 City with Best Sales (USD)", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10_city_inv_graph(df_city_inv):    
    ax = df_city_inv.plot.barh(figsize=(7, 4))
    
    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("City Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Number of invoices", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 City with Best number of invoices", fontsize=13, color='r', pad=15);

#### <span style="color:blue">Top 10 counties with Best sale/invoices</span>

In [ ]:
def top10_counties_sale_inv(df):
    county_sale = df.groupby(['County'])['Sale (Dollars)'].sum().nlargest(10)                                   
    county_sale_fr = county_sale.apply(lambda x: f'${x:,.0f}').to_frame()

    county_inv = df['County'].value_counts().nlargest(10)
    county_inv_fr = county_inv.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoices')

    return county_sale, county_sale_fr, county_inv, county_inv_fr

In [ ]:
def plot_top10_county_sale_graph(df_county_sale):
    ax = df_county_sale.plot.barh(figsize=(7, 4))

    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("County Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Sales (USD)", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 County with Best Sales (USD)", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10_county_inv_graph(df_county_inv):
    ax = df_county_inv.plot.barh(figsize=(7, 4))
    
    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("County Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Number of invoices", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 County with Best number of invoices", fontsize=13, color='r', pad=15);

#### <span style="color:blue">Top 10 categories with Best sale/invoices</span>

In [ ]:
def top10_cat_sale_inv(df):
    cat_sale = df.groupby(['Category Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
    cat_sale_fr = cat_sale.apply(lambda x: f'${x:,.0f}').to_frame()

    cat_inv = df['Category Name'].value_counts().nlargest(10)
    cat_inv_fr = cat_inv.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoices')

    return cat_sale, cat_sale_fr, cat_inv, cat_inv_fr

In [ ]:
def plot_top10cat_sale_graph(df_cat_sale):
    ax = df_cat_sale.plot.barh(figsize=(7, 4))

    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("Category Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Sales (USD)", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 Categories with Best Sales (USD)", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10cat_inv_graph(df_cat_inv):
    ax = df_cat_inv.plot.barh(figsize=(7, 4))
    
    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("Categories Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Number of invoices", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 Categories with Best number of invoices", fontsize=13, color='r', pad=15);

#### <span style="color:blue">Top 10 vendors with Best sale/invoices</span>

In [ ]:
def top10_vendors_sale_inv(df):
    vendor_sale = df.groupby(['Vendor Name'])['Sale (Dollars)'].sum().nlargest(10)                                   
    vendor_sale_fr = vendor_sale.apply(lambda x: f'${x:,.0f}').to_frame()

    vendor_inv = df['Vendor Name'].value_counts().nlargest(10)
    vendor_inv_fr = vendor_inv.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoices')

    return vendor_sale, vendor_sale_fr, vendor_inv, vendor_inv_fr

In [ ]:
def plot_top10_vendors_sale_graph(df_vendor_sale):
    ax = df_vendor_sale.plot.barh(figsize=(7, 4))

    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("Vendor Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Sales (USD)", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 Vendors with Best Sales (USD)", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10_vendors_inv_graph(df_vendor_inv):
    ax = df_vendor_inv.plot.barh(figsize=(7, 4))
    
    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("Vendor Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Number of invoices", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 Vendors with Best number of invoices", fontsize=13, color='r', pad=15);

#### <span style="color:blue">Top 10 items with Best sale/invoices</span>

In [ ]:
def top10_items_sale_inv(df):
    item_sale = df.groupby(['Item Description'])['Sale (Dollars)'].sum().nlargest(10)                                   
    item_sale_fr = item_sale.apply(lambda x: f'${x:,.0f}').to_frame()

    item_inv = df['Item Description'].value_counts().nlargest(10)
    item_inv_fr = item_inv.apply(lambda x: f'{x:,.0f}').to_frame(name='Num of invoices')

    return item_sale, item_sale_fr, item_inv, item_inv_fr

In [ ]:
def plot_top10_items_sale_graph(df_item_sale):
    ax = df_item_sale.plot.barh(figsize=(7, 4))

    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("Item Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Sales (USD)", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 Items with Best Sales (USD)", fontsize=13, color='r', pad=15);

In [ ]:
def plot_top10_items_inv_graph(df_item_inv):
    ax = df_item_inv.plot.barh(figsize=(7, 4))
    
    ax.xaxis.set_major_formatter(reformat_large_tick_values)
    ax.xaxis.set_minor_formatter(NullFormatter())

    plt.ylabel("Item Name", fontsize=13, color='r', labelpad=15)
    plt.xlabel("Number of invoices", fontsize=13, color='r', labelpad=15)
    plt.title("Top 10 Items with Best number of invoices", fontsize=13, color='r', pad=15);

# <span style="color:red">5. Data Visualzation</span>

## <span style="color:blue">Overall Report 2013</span>

In [ ]:
total_save, total_inv = total_sale_invoice(df_2013)
print(f"Total sale: {total_save}")
print(f"Total invoice: {total_inv}")

In [ ]:
month_sale, month_inv, sale_invoice_month = total_sale_invoice_month(df_2013)

print(f"Total sales, invoices of Months:\n {sale_invoice_month}")
plot_month_sale_inv_graph(month_sale, month_inv)

In [ ]:
dow_sale, dow_inv, sale_invoice_dow = total_sale_invoice_dow(df_2013)

print(f"Total sales, invoices of Day of Week:\n {sale_invoice_dow}")
plot_dow_sale_inv_graph(dow_sale, dow_inv)

In [ ]:
store_sale, store_sale_fr, store_inv, store_inv_fr = top10_store_sale_inv(df_2013)

In [ ]:
print(f"Top 10 stores with best sale:\n {store_sale_fr}")
plot_top10_store_sale_graph(store_sale)

In [ ]:
print(f"\nTop 10 stores with best invoices:\n {store_inv_fr}")
plot_top10_store_inv_graph(store_inv)

In [ ]:
county_sale, county_sale_fr, county_inv, county_inv_fr = top10_counties_sale_inv(df_2013)

In [ ]:
print(f"Top 10 counties with best sale:\n {county_sale_fr}")
plot_top10_county_sale_graph(county_sale)

In [ ]:
print(f"\nTop 10 counties with best invoices:\n {county_inv_fr}")
plot_top10_county_inv_graph(county_inv)

In [ ]:
city_sale, city_sale_fr, city_inv, city_inv_fr = top10_cities_sale_inv(df_2013)

In [ ]:
print(f"Top 10 cities with best sale:\n {city_sale_fr}")
plot_top10_city_sale_graph(city_sale)

In [ ]:
print(f"\nTop 10 cities with best invoices:\n {city_inv_fr}")
plot_top10_city_inv_graph(city_inv)

In [ ]:
cat_sale, cat_sale_fr, cat_inv, cat_inv_fr = top10_cat_sale_inv(df_2013)

In [ ]:
print(f"Top 10 categories with best sale:\n {cat_sale_fr}")
plot_top10cat_sale_graph(cat_sale)

In [ ]:
print(f"\nTop 10 categories with best invoices:\n {cat_inv_fr}")
plot_top10cat_inv_graph(cat_inv)

In [ ]:
vendor_sale, vendor_sale_fr, vendor_inv, vendor_inv_fr = top10_cat_sale_inv(df_2013)

In [ ]:
print(f"Top 10 vendors with best sale:\n {vendor_sale_fr}")
plot_top10_vendors_sale_graph(vendor_sale)

In [ ]:
print(f"\nTop 10 vendors with best invoices:\n {vendor_inv_fr}")
plot_top10_vendors_inv_graph(vendor_inv)

In [ ]:
item_sale, item_sale_fr, item_inv, item_inv_fr = top10_cat_sale_inv(df_2013)

In [ ]:
print(f"Top 10 items with best sale:\n {item_sale_fr}")
plot_top10_items_sale_graph(item_sale)

In [ ]:
print(f"\nTop 10 items with best invoices:\n {item_inv_fr}")
plot_top10_items_sale_graph(item_inv)

## <span style="color:blue">Filter by a Single Date in 2013</span>

In [ ]:
input_date = '2013-01-02'
full_date = input_date + ' 00:00:00'

if exist_date(df_2013, full_date):
    df_bySingleDate_2013 = df_BySingleDate(df_2013, input_date)

In [ ]:
total_save, total_inv = total_sale_invoice(df_bySingleDate_2013)
print(f"Total sale: {total_save}")
print(f"Total invoice: {total_inv}")

In [ ]:
store_sale, store_sale_fr, store_inv, store_inv_fr = top10_store_sale_inv(df_bySingleDate_2013)

In [ ]:
print(f"Top 10 stores with best sale:\n {store_sale_fr}")
plot_top10_store_sale_graph(store_sale)

In [ ]:
print(f"\nTop 10 stores with best invoices:\n {store_inv_fr}")
plot_top10_store_inv_graph(store_inv)

In [ ]:
county_sale, county_sale_fr, county_inv, county_inv_fr = top10_counties_sale_inv(df_bySingleDate_2013)

In [ ]:
print(f"Top 10 counties with best sale:\n {county_sale_fr}")
plot_top10_county_sale_graph(county_sale)

In [ ]:
print(f"\nTop 10 counties with best invoices:\n {county_inv_fr}")
plot_top10_county_inv_graph(county_inv)

In [ ]:
city_sale, city_sale_fr, city_inv, city_inv_fr = top10_cities_sale_inv(df_bySingleDate_2013)

In [ ]:
print(f"Top 10 cities with best sale:\n {city_sale_fr}")
plot_top10_city_sale_graph(city_sale)

In [ ]:
print(f"\nTop 10 cities with best invoices:\n {city_inv_fr}")
plot_top10_city_inv_graph(city_inv)

In [ ]:
cat_sale, cat_sale_fr, cat_inv, cat_inv_fr = top10_cat_sale_inv(df_bySingleDate_2013)

In [ ]:
print(f"Top 10 categories with best sale:\n {cat_sale_fr}")
plot_top10cat_sale_graph(cat_sale)

In [ ]:
print(f"\nTop 10 categories with best invoices:\n {cat_inv_fr}")
plot_top10cat_inv_graph(cat_inv)

In [ ]:
vendor_sale, vendor_sale_fr, vendor_inv, vendor_inv_fr = top10_cat_sale_inv(df_bySingleDate_2013)

In [ ]:
print(f"Top 10 vendors with best sale:\n {vendor_sale_fr}")
plot_top10_vendors_sale_graph(vendor_sale)

In [ ]:
print(f"\nTop 10 vendors with best invoices:\n {vendor_inv_fr}")
plot_top10_vendors_inv_graph(vendor_inv)

In [ ]:
item_sale, item_sale_fr, item_inv, item_inv_fr = top10_cat_sale_inv(df_bySingleDate_2013)

In [ ]:
print(f"Top 10 items with best sale:\n {item_sale_fr}")
plot_top10_items_sale_graph(item_sale)

In [ ]:
print(f"\nTop 10 items with best invoices:\n {item_inv_fr}")
plot_top10_items_sale_graph(item_inv)

## <span style="color:blue">Filter by Dates in 2013</span>

In [ ]:
start_date = "2013-01-03"
end_date = "2013-01-10"

In [ ]:
df_ByDates_2013 = df_ByDates(df_2013, start_date, end_date)

In [ ]:
lst_missing_dates = find_missing_dates(df_ByDates_2013, start_date, end_date)
lst_missing_dates_name = lst_missing_dates.day_name()

print(f"{lst_missing_dates}\n")
print(lst_missing_dates_name)

In [ ]:
total_save, total_inv = total_sale_invoice(df_ByDates_2013)
print(f"Total sale: {total_save}")
print(f"Total invoice: {total_inv}")

In [ ]:
dow_sale, dow_inv, sale_invoice_dow = total_sale_invoice_dow(df_ByDates_2013)

print(f"Total sales, invoices of Day of Week:\n {sale_invoice_dow}")
plot_dow_sale_inv_graph(dow_sale, dow_inv)

In [ ]:
store_sale, store_sale_fr, store_inv, store_inv_fr = top10_store_sale_inv(df_ByDates_2013)

In [ ]:
print(f"Top 10 stores with best sale:\n {store_sale_fr}")
plot_top10_store_sale_graph(store_sale)

In [ ]:
print(f"\nTop 10 stores with best invoices:\n {store_inv_fr}")
plot_top10_store_inv_graph(store_inv)

In [ ]:
county_sale, county_sale_fr, county_inv, county_inv_fr = top10_counties_sale_inv(df_ByDates_2013)

In [ ]:
print(f"Top 10 counties with best sale:\n {county_sale_fr}")
plot_top10_county_sale_graph(county_sale)

In [ ]:
print(f"\nTop 10 counties with best invoices:\n {county_inv_fr}")
plot_top10_county_inv_graph(county_inv)

In [ ]:
city_sale, city_sale_fr, city_inv, city_inv_fr = top10_cities_sale_inv(df_ByDates_2013)

In [ ]:
print(f"Top 10 cities with best sale:\n {city_sale_fr}")
plot_top10_city_sale_graph(city_sale)

In [ ]:
print(f"\nTop 10 cities with best invoices:\n {city_inv_fr}")
plot_top10_city_inv_graph(city_inv)

In [ ]:
cat_sale, cat_sale_fr, cat_inv, cat_inv_fr = top10_cat_sale_inv(df_ByDates_2013)

In [ ]:
print(f"Top 10 categories with best sale:\n {cat_sale_fr}")
plot_top10cat_sale_graph(cat_sale)

In [ ]:
print(f"\nTop 10 categories with best invoices:\n {cat_inv_fr}")
plot_top10cat_inv_graph(cat_inv)

In [ ]:
vendor_sale, vendor_sale_fr, vendor_inv, vendor_inv_fr = top10_cat_sale_inv(df_ByDates_2013)

In [ ]:
print(f"Top 10 vendors with best sale:\n {vendor_sale_fr}")
plot_top10_vendors_sale_graph(vendor_sale)

In [ ]:
print(f"\nTop 10 vendors with best invoices:\n {vendor_inv_fr}")
plot_top10_vendors_inv_graph(vendor_inv)

In [ ]:
item_sale, item_sale_fr, item_inv, item_inv_fr = top10_cat_sale_inv(df_ByDates_2013)

In [ ]:
print(f"Top 10 items with best sale:\n {item_sale_fr}")
plot_top10_items_sale_graph(item_sale)

In [ ]:
print(f"\nTop 10 items with best invoices:\n {item_inv_fr}")
plot_top10_items_sale_graph(item_inv)

## <span style="color:blue">Filter by Store Name, Store Number in 2013</span>

In [ ]:
store_number = "2549"

In [ ]:
if exist_inputData(store_number):
    df_ByStore_2013 = df_ByStoreNumber(df_2013, store_number)

In [ ]:
total_save, total_inv = total_sale_invoice(df_ByStore_2013)
print(f"Total sale: {total_save}")
print(f"Total invoice: {total_inv}")

In [ ]:
month_sale, month_inv, sale_invoice_month = total_sale_invoice_month(df_ByStore_2013)

print(f"Total sales, invoices of Months:\n {sale_invoice_month}")
plot_month_sale_inv_graph(month_sale, month_inv)

In [ ]:
dow_sale, dow_inv, sale_invoice_dow = total_sale_invoice_dow(df_ByStore_2013)

print(f"Total sales, invoices of Day of Week:\n {sale_invoice_dow}")
plot_dow_sale_inv_graph(dow_sale, dow_inv)

In [ ]:
store_sale, store_sale_fr, store_inv, store_inv_fr = top10_store_sale_inv(df_ByStore_2013)

In [ ]:
print(f"Top 10 stores with best sale:\n {store_sale_fr}")
plot_top10_store_sale_graph(store_sale)

In [ ]:
print(f"\nTop 10 stores with best invoices:\n {store_inv_fr}")
plot_top10_store_inv_graph(store_inv)

In [ ]:
county_sale, county_sale_fr, county_inv, county_inv_fr = top10_counties_sale_inv(df_ByStore_2013)

In [ ]:
print(f"Top 10 counties with best sale:\n {county_sale_fr}")
plot_top10_county_sale_graph(county_sale)

In [ ]:
print(f"\nTop 10 counties with best invoices:\n {county_inv_fr}")
plot_top10_county_inv_graph(county_inv)

In [ ]:
city_sale, city_sale_fr, city_inv, city_inv_fr = top10_cities_sale_inv(df_ByStore_2013)

In [ ]:
print(f"Top 10 cities with best sale:\n {city_sale_fr}")
plot_top10_city_sale_graph(city_sale)

In [ ]:
print(f"\nTop 10 cities with best invoices:\n {city_inv_fr}")
plot_top10_city_inv_graph(city_inv)

In [ ]:
cat_sale, cat_sale_fr, cat_inv, cat_inv_fr = top10_cat_sale_inv(df_ByStore_2013)

In [ ]:
print(f"Top 10 categories with best sale:\n {cat_sale_fr}")
plot_top10cat_sale_graph(cat_sale)

In [ ]:
print(f"\nTop 10 categories with best invoices:\n {cat_inv_fr}")
plot_top10cat_inv_graph(cat_inv)

In [ ]:
vendor_sale, vendor_sale_fr, vendor_inv, vendor_inv_fr = top10_cat_sale_inv(df_ByStore_2013)

In [ ]:
print(f"Top 10 vendors with best sale:\n {vendor_sale_fr}")
plot_top10_vendors_sale_graph(vendor_sale)

In [ ]:
print(f"\nTop 10 vendors with best invoices:\n {vendor_inv_fr}")
plot_top10_vendors_inv_graph(vendor_inv)

In [ ]:
item_sale, item_sale_fr, item_inv, item_inv_fr = top10_cat_sale_inv(df_ByStore_2013)

In [ ]:
print(f"Top 10 items with best sale:\n {item_sale_fr}")
plot_top10_items_sale_graph(item_sale)

In [ ]:
print(f"\nTop 10 items with best invoices:\n {item_inv_fr}")
plot_top10_items_sale_graph(item_inv)

## <span style="color:blue">Filter by City Name in 2013</span>

In [ ]:
city_name = "INDIANOLA"

In [ ]:
if exist_inputData(city_name):
    df_ByCity_2013 = df_ByCity(df_2013, city_name)

In [ ]:
total_save, total_inv = total_sale_invoice(df_ByCity_2013)
print(f"Total sale: {total_save}")
print(f"Total invoice: {total_inv}")

In [ ]:
month_sale, month_inv, sale_invoice_month = total_sale_invoice_month(df_ByCity_2013)

print(f"Total sales, invoices of Months:\n {sale_invoice_month}")
plot_month_sale_inv_graph(month_sale, month_inv)

In [ ]:
dow_sale, dow_inv, sale_invoice_dow = total_sale_invoice_dow(df_ByCity_2013)

print(f"Total sales, invoices of Day of Week:\n {sale_invoice_dow}")
plot_dow_sale_inv_graph(dow_sale, dow_inv)

In [ ]:
store_sale, store_sale_fr, store_inv, store_inv_fr = top10_store_sale_inv(df_ByCity_2013)

In [ ]:
print(f"Top 10 stores with best sale:\n {store_sale_fr}")
plot_top10_store_sale_graph(store_sale)

In [ ]:
print(f"\nTop 10 stores with best invoices:\n {store_inv_fr}")
plot_top10_store_inv_graph(store_inv)

In [ ]:
county_sale, county_sale_fr, county_inv, county_inv_fr = top10_counties_sale_inv(df_ByCity_2013)

In [ ]:
print(f"Top 10 counties with best sale:\n {county_sale_fr}")
plot_top10_county_sale_graph(county_sale)

In [ ]:
print(f"\nTop 10 counties with best invoices:\n {county_inv_fr}")
plot_top10_county_inv_graph(county_inv)

In [ ]:
city_sale, city_sale_fr, city_inv, city_inv_fr = top10_cities_sale_inv(df_ByCity_2013)

In [ ]:
print(f"Top 10 cities with best sale:\n {city_sale_fr}")
plot_top10_city_sale_graph(city_sale)

In [ ]:
print(f"\nTop 10 cities with best invoices:\n {city_inv_fr}")
plot_top10_city_inv_graph(city_inv)

In [ ]:
cat_sale, cat_sale_fr, cat_inv, cat_inv_fr = top10_cat_sale_inv(df_ByCity_2013)

In [ ]:
print(f"Top 10 categories with best sale:\n {cat_sale_fr}")
plot_top10cat_sale_graph(cat_sale)

In [ ]:
print(f"\nTop 10 categories with best invoices:\n {cat_inv_fr}")
plot_top10cat_inv_graph(cat_inv)

In [ ]:
vendor_sale, vendor_sale_fr, vendor_inv, vendor_inv_fr = top10_cat_sale_inv(df_ByCity_2013)

In [ ]:
print(f"Top 10 vendors with best sale:\n {vendor_sale_fr}")
plot_top10_vendors_sale_graph(vendor_sale)

In [ ]:
print(f"\nTop 10 vendors with best invoices:\n {vendor_inv_fr}")
plot_top10_vendors_inv_graph(vendor_inv)

In [ ]:
item_sale, item_sale_fr, item_inv, item_inv_fr = top10_cat_sale_inv(df_ByCity_2013)

In [ ]:
print(f"Top 10 items with best sale:\n {item_sale_fr}")
plot_top10_items_sale_graph(item_sale)

In [ ]:
print(f"\nTop 10 items with best invoices:\n {item_inv_fr}")
plot_top10_items_sale_graph(item_inv)

## <span style="color:blue">Filter by County Name in 2013</span>

In [ ]:
county_name = "POLK"

In [ ]:
if exist_inputData(county_name):
    df_ByCounty_2013 = df_ByCounty(df_2013, county_name)

In [ ]:
total_save, total_inv = total_sale_invoice(df_ByCounty_2013)
print(f"Total sale: {total_save}")
print(f"Total invoice: {total_inv}")

In [ ]:
month_sale, month_inv, sale_invoice_month = total_sale_invoice_month(df_ByCounty_2013)

print(f"Total sales, invoices of Months:\n {sale_invoice_month}")
plot_month_sale_inv_graph(month_sale, month_inv)

In [ ]:
dow_sale, dow_inv, sale_invoice_dow = total_sale_invoice_dow(df_ByCounty_2013)

print(f"Total sales, invoices of Day of Week:\n {sale_invoice_dow}")
plot_dow_sale_inv_graph(dow_sale, dow_inv)

In [ ]:
store_sale, store_sale_fr, store_inv, store_inv_fr = top10_store_sale_inv(df_ByCounty_2013)

In [ ]:
print(f"Top 10 stores with best sale:\n {store_sale_fr}")
plot_top10_store_sale_graph(store_sale)

In [ ]:
print(f"\nTop 10 stores with best invoices:\n {store_inv_fr}")
plot_top10_store_inv_graph(store_inv)

In [ ]:
county_sale, county_sale_fr, county_inv, county_inv_fr = top10_counties_sale_inv(df_ByCounty_2013)

In [ ]:
print(f"Top 10 counties with best sale:\n {county_sale_fr}")
plot_top10_county_sale_graph(county_sale)

In [ ]:
print(f"\nTop 10 counties with best invoices:\n {county_inv_fr}")
plot_top10_county_inv_graph(county_inv)

In [ ]:
city_sale, city_sale_fr, city_inv, city_inv_fr = top10_cities_sale_inv(df_ByCounty_2013)

In [ ]:
print(f"Top 10 cities with best sale:\n {city_sale_fr}")
plot_top10_city_sale_graph(city_sale)

In [ ]:
print(f"\nTop 10 cities with best invoices:\n {city_inv_fr}")
plot_top10_city_inv_graph(city_inv)

In [ ]:
cat_sale, cat_sale_fr, cat_inv, cat_inv_fr = top10_cat_sale_inv(df_ByCounty_2013)

In [ ]:
print(f"Top 10 categories with best sale:\n {cat_sale_fr}")
plot_top10cat_sale_graph(cat_sale)

In [ ]:
print(f"\nTop 10 categories with best invoices:\n {cat_inv_fr}")
plot_top10cat_inv_graph(cat_inv)

In [ ]:
vendor_sale, vendor_sale_fr, vendor_inv, vendor_inv_fr = top10_cat_sale_inv(df_ByCounty_2013)

In [ ]:
print(f"Top 10 vendors with best sale:\n {vendor_sale_fr}")
plot_top10_vendors_sale_graph(vendor_sale)

In [ ]:
print(f"\nTop 10 vendors with best invoices:\n {vendor_inv_fr}")
plot_top10_vendors_inv_graph(vendor_inv)

In [ ]:
item_sale, item_sale_fr, item_inv, item_inv_fr = top10_cat_sale_inv(df_ByCounty_2013)

In [ ]:
print(f"Top 10 items with best sale:\n {item_sale_fr}")
plot_top10_items_sale_graph(item_sale)

In [ ]:
print(f"\nTop 10 items with best invoices:\n {item_inv_fr}")
plot_top10_items_sale_graph(item_inv)

## <span style="color:blue">Filter by Cat ID in 2013</span>

In [ ]:
cat_id = "1062300.0"

In [ ]:
if exist_inputData(cat_id):
    df_ByCat_2013 = df_ByCat(df_2013, cat_id)

In [ ]:
total_save, total_inv = total_sale_invoice(df_ByCat_2013)
print(f"Total sale: {total_save}")
print(f"Total invoice: {total_inv}")

In [ ]:
month_sale, month_inv, sale_invoice_month = total_sale_invoice_month(df_ByCat_2013)

print(f"Total sales, invoices of Months:\n {sale_invoice_month}")
plot_month_sale_inv_graph(month_sale, month_inv)

In [ ]:
dow_sale, dow_inv, sale_invoice_dow = total_sale_invoice_dow(df_ByCat_2013)

print(f"Total sales, invoices of Day of Week:\n {sale_invoice_dow}")
plot_dow_sale_inv_graph(dow_sale, dow_inv)

In [ ]:
store_sale, store_sale_fr, store_inv, store_inv_fr = top10_store_sale_inv(df_ByCat_2013)

In [ ]:
print(f"Top 10 stores with best sale:\n {store_sale_fr}")
plot_top10_store_sale_graph(store_sale)

In [ ]:
print(f"\nTop 10 stores with best invoices:\n {store_inv_fr}")
plot_top10_store_inv_graph(store_inv)

In [ ]:
county_sale, county_sale_fr, county_inv, county_inv_fr = top10_counties_sale_inv(df_ByCat_2013)

In [ ]:
print(f"Top 10 counties with best sale:\n {county_sale_fr}")
plot_top10_county_sale_graph(county_sale)

In [ ]:
print(f"\nTop 10 counties with best invoices:\n {county_inv_fr}")
plot_top10_county_inv_graph(county_inv)

In [ ]:
city_sale, city_sale_fr, city_inv, city_inv_fr = top10_cities_sale_inv(df_ByCat_2013)

In [ ]:
print(f"Top 10 cities with best sale:\n {city_sale_fr}")
plot_top10_city_sale_graph(city_sale)

In [ ]:
print(f"\nTop 10 cities with best invoices:\n {city_inv_fr}")
plot_top10_city_inv_graph(city_inv)

In [ ]:
cat_sale, cat_sale_fr, cat_inv, cat_inv_fr = top10_cat_sale_inv(df_ByCat_2013)

In [ ]:
print(f"Top 10 categories with best sale:\n {cat_sale_fr}")
plot_top10cat_sale_graph(cat_sale)

In [ ]:
print(f"\nTop 10 categories with best invoices:\n {cat_inv_fr}")
plot_top10cat_inv_graph(cat_inv)

In [ ]:
vendor_sale, vendor_sale_fr, vendor_inv, vendor_inv_fr = top10_cat_sale_inv(df_ByCat_2013)

In [ ]:
print(f"Top 10 vendors with best sale:\n {vendor_sale_fr}")
plot_top10_vendors_sale_graph(vendor_sale)

In [ ]:
print(f"\nTop 10 vendors with best invoices:\n {vendor_inv_fr}")
plot_top10_vendors_inv_graph(vendor_inv)

In [ ]:
item_sale, item_sale_fr, item_inv, item_inv_fr = top10_cat_sale_inv(df_ByCat_2013)

In [ ]:
print(f"Top 10 items with best sale:\n {item_sale_fr}")
plot_top10_items_sale_graph(item_sale)

In [ ]:
print(f"\nTop 10 items with best invoices:\n {item_inv_fr}")
plot_top10_items_sale_graph(item_inv)

## <span style="color:blue">Filter by Vendor in 2013</span>

In [ ]:
vendor_num = "370"

In [ ]:
if exist_inputData(vendor_num):
    df_ByVendor_2013 = df_ByVendor(df_2013, vendor_num)

In [ ]:
total_save, total_inv = total_sale_invoice(df_ByVendor_2013)
print(f"Total sale: {total_save}")
print(f"Total invoice: {total_inv}")

In [ ]:
month_sale, month_inv, sale_invoice_month = total_sale_invoice_month(df_ByVendor_2013)

print(f"Total sales, invoices of Months:\n {sale_invoice_month}")
plot_month_sale_inv_graph(month_sale, month_inv)

In [ ]:
dow_sale, dow_inv, sale_invoice_dow = total_sale_invoice_dow(df_ByVendor_2013)

print(f"Total sales, invoices of Day of Week:\n {sale_invoice_dow}")
plot_dow_sale_inv_graph(dow_sale, dow_inv)

In [ ]:
store_sale, store_sale_fr, store_inv, store_inv_fr = top10_store_sale_inv(df_ByVendor_2013)

In [ ]:
print(f"Top 10 stores with best sale:\n {store_sale_fr}")
plot_top10_store_sale_graph(store_sale)

In [ ]:
print(f"\nTop 10 stores with best invoices:\n {store_inv_fr}")
plot_top10_store_inv_graph(store_inv)

In [ ]:
county_sale, county_sale_fr, county_inv, county_inv_fr = top10_counties_sale_inv(df_ByVendor_2013)

In [ ]:
print(f"Top 10 counties with best sale:\n {county_sale_fr}")
plot_top10_county_sale_graph(county_sale)

In [ ]:
print(f"\nTop 10 counties with best invoices:\n {county_inv_fr}")
plot_top10_county_inv_graph(county_inv)

In [ ]:
city_sale, city_sale_fr, city_inv, city_inv_fr = top10_cities_sale_inv(df_ByVendor_2013)

In [ ]:
print(f"Top 10 cities with best sale:\n {city_sale_fr}")
plot_top10_city_sale_graph(city_sale)

In [ ]:
print(f"\nTop 10 cities with best invoices:\n {city_inv_fr}")
plot_top10_city_inv_graph(city_inv)

In [ ]:
cat_sale, cat_sale_fr, cat_inv, cat_inv_fr = top10_cat_sale_inv(df_ByVendor_2013)

In [ ]:
print(f"Top 10 categories with best sale:\n {cat_sale_fr}")
plot_top10cat_sale_graph(cat_sale)

In [ ]:
print(f"\nTop 10 categories with best invoices:\n {cat_inv_fr}")
plot_top10cat_inv_graph(cat_inv)

In [ ]:
vendor_sale, vendor_sale_fr, vendor_inv, vendor_inv_fr = top10_cat_sale_inv(df_ByVendor_2013)

In [ ]:
print(f"Top 10 vendors with best sale:\n {vendor_sale_fr}")
plot_top10_vendors_sale_graph(vendor_sale)

In [ ]:
print(f"\nTop 10 vendors with best invoices:\n {vendor_inv_fr}")
plot_top10_vendors_inv_graph(vendor_inv)

In [ ]:
item_sale, item_sale_fr, item_inv, item_inv_fr = top10_cat_sale_inv(df_ByVendor_2013)

In [ ]:
print(f"Top 10 items with best sale:\n {item_sale_fr}")
plot_top10_items_sale_graph(item_sale)

In [ ]:
print(f"\nTop 10 items with best invoices:\n {item_inv_fr}")
plot_top10_items_sale_graph(item_inv)

## <span style="color:blue">Filter by Item in 2013</span>

In [ ]:
item_num = "42717"

In [ ]:
if exist_inputData(item_num):
    df_ByItem_2013 = df_ByItem(df_2013, item_num)

In [ ]:
total_save, total_inv = total_sale_invoice(df_ByItem_2013)
print(f"Total sale: {total_save}")
print(f"Total invoice: {total_inv}")

In [ ]:
month_sale, month_inv, sale_invoice_month = total_sale_invoice_month(df_ByItem_2013)

print(f"Total sales, invoices of Months:\n {sale_invoice_month}")
plot_month_sale_inv_graph(month_sale, month_inv)

In [ ]:
dow_sale, dow_inv, sale_invoice_dow = total_sale_invoice_dow(df_ByItem_2013)

print(f"Total sales, invoices of Day of Week:\n {sale_invoice_dow}")
plot_dow_sale_inv_graph(dow_sale, dow_inv)

In [ ]:
store_sale, store_sale_fr, store_inv, store_inv_fr = top10_store_sale_inv(df_ByItem_2013)

In [ ]:
print(f"Top 10 stores with best sale:\n {store_sale_fr}")
plot_top10_store_sale_graph(store_sale)

In [ ]:
print(f"\nTop 10 stores with best invoices:\n {store_inv_fr}")
plot_top10_store_inv_graph(store_inv)

In [ ]:
county_sale, county_sale_fr, county_inv, county_inv_fr = top10_counties_sale_inv(df_ByItem_2013)

In [ ]:
print(f"Top 10 counties with best sale:\n {county_sale_fr}")
plot_top10_county_sale_graph(county_sale)

In [ ]:
print(f"\nTop 10 counties with best invoices:\n {county_inv_fr}")
plot_top10_county_inv_graph(county_inv)

In [ ]:
city_sale, city_sale_fr, city_inv, city_inv_fr = top10_cities_sale_inv(df_ByItem_2013)

In [ ]:
print(f"Top 10 cities with best sale:\n {city_sale_fr}")
plot_top10_city_sale_graph(city_sale)

In [ ]:
print(f"\nTop 10 cities with best invoices:\n {city_inv_fr}")
plot_top10_city_inv_graph(city_inv)

In [ ]:
cat_sale, cat_sale_fr, cat_inv, cat_inv_fr = top10_cat_sale_inv(df_ByItem_2013)

In [ ]:
print(f"Top 10 categories with best sale:\n {cat_sale_fr}")
plot_top10cat_sale_graph(cat_sale)

In [ ]:
print(f"\nTop 10 categories with best invoices:\n {cat_inv_fr}")
plot_top10cat_inv_graph(cat_inv)

In [ ]:
vendor_sale, vendor_sale_fr, vendor_inv, vendor_inv_fr = top10_cat_sale_inv(df_ByItem_2013)

In [ ]:
print(f"Top 10 vendors with best sale:\n {vendor_sale_fr}")
plot_top10_vendors_sale_graph(vendor_sale)

In [ ]:
print(f"\nTop 10 vendors with best invoices:\n {vendor_inv_fr}")
plot_top10_vendors_inv_graph(vendor_inv)

In [ ]:
item_sale, item_sale_fr, item_inv, item_inv_fr = top10_cat_sale_inv(df_ByItem_2013)

In [ ]:
print(f"Top 10 items with best sale:\n {item_sale_fr}")
plot_top10_items_sale_graph(item_sale)

In [ ]:
print(f"\nTop 10 items with best invoices:\n {item_inv_fr}")
plot_top10_items_sale_graph(item_inv)

## Filter by a Single Month

In [ ]:
month = "2013-01"

In [ ]:
df_BySingleMonth_2013 = df_BySingleMonth(df_2013, month)

In [ ]:
total_save, total_inv = total_sale_invoice(df_BySingleMonth_2013)
print(f"Total sale: {total_save}")
print(f"Total invoice: {total_inv}")

In [ ]:
dow_sale, dow_inv, sale_invoice_dow = total_sale_invoice_dow(df_BySingleMonth_2013)

print(f"Total sales, invoices of Day of Week:\n {sale_invoice_dow}")
plot_dow_sale_inv_graph(dow_sale, dow_inv)

In [ ]:
store_sale, store_sale_fr, store_inv, store_inv_fr = top10_store_sale_inv(df_BySingleMonth_2013)

In [ ]:
print(f"Top 10 stores with best sale:\n {store_sale_fr}")
plot_top10_store_sale_graph(store_sale)

In [ ]:
print(f"\nTop 10 stores with best invoices:\n {store_inv_fr}")
plot_top10_store_inv_graph(store_inv)

In [ ]:
county_sale, county_sale_fr, county_inv, county_inv_fr = top10_counties_sale_inv(df_BySingleMonth_2013)

In [ ]:
print(f"Top 10 counties with best sale:\n {county_sale_fr}")
plot_top10_county_sale_graph(county_sale)

In [ ]:
print(f"\nTop 10 counties with best invoices:\n {county_inv_fr}")
plot_top10_county_inv_graph(county_inv)

In [ ]:
city_sale, city_sale_fr, city_inv, city_inv_fr = top10_cities_sale_inv(df_BySingleMonth_2013)

In [ ]:
print(f"Top 10 cities with best sale:\n {city_sale_fr}")
plot_top10_city_sale_graph(city_sale)

In [ ]:
print(f"\nTop 10 cities with best invoices:\n {city_inv_fr}")
plot_top10_city_inv_graph(city_inv)

In [ ]:
cat_sale, cat_sale_fr, cat_inv, cat_inv_fr = top10_cat_sale_inv(df_BySingleMonth_2013)

In [ ]:
print(f"Top 10 categories with best sale:\n {cat_sale_fr}")
plot_top10cat_sale_graph(cat_sale)

In [ ]:
print(f"\nTop 10 categories with best invoices:\n {cat_inv_fr}")
plot_top10cat_inv_graph(cat_inv)

In [ ]:
vendor_sale, vendor_sale_fr, vendor_inv, vendor_inv_fr = top10_cat_sale_inv(df_BySingleMonth_2013)

In [ ]:
print(f"Top 10 vendors with best sale:\n {vendor_sale_fr}")
plot_top10_vendors_sale_graph(vendor_sale)

In [ ]:
print(f"\nTop 10 vendors with best invoices:\n {vendor_inv_fr}")
plot_top10_vendors_inv_graph(vendor_inv)

In [ ]:
item_sale, item_sale_fr, item_inv, item_inv_fr = top10_cat_sale_inv(df_BySingleMonth_2013)

In [ ]:
print(f"Top 10 items with best sale:\n {item_sale_fr}")
plot_top10_items_sale_graph(item_sale)

In [ ]:
print(f"\nTop 10 items with best invoices:\n {item_inv_fr}")
plot_top10_items_sale_graph(item_inv)